In [1]:

from sklearn.model_selection import train_test_split
from ast import literal_eval

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from torch import nn
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour
import torch
from torch.utils.data import DataLoader
from transformers import BertModel, AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import re
import tensorflow as tf

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import glob
file_names = glob.glob('/content/drive/MyDrive/GA Research Project - Covid Tweet Classification/Twitter_Covid_GAProject/Annotated Data/*.csv')
print(file_names)

Mounted at /content/drive
[]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df_data = []
for file_name in file_names:
    temp = pd.read_csv(file_name)
    if 'CDCFLu' in file_name:
        temp = temp.head(50)
    else:
        temp = temp.head(499)
    df_data.append(temp)

In [ ]:
df_data = pd.concat(df_data)

In [ ]:
print(df_data.shape)

(2046, 13)


In [ ]:
print(df_data["Category"].unique())

[nan 'Outbreak' 'Vaccine; Outbreak' 'Outbreak; Prevention; Vaccine'
 'Outbreak; Vaccine' 'Outbreak; Symptoms' 'Outbreak; Prevention' 'Vaccine'
 'Prevention' 'Outbreak; Vaccine; Prevention' 'Vaccine; Prevention'
 'Vaccine, Prevention; Symptoms' 'Vaccine, Outbreak' 'Travel; Outbreak'
 'Travel' 'Prevention, Travel' 'Travel; Prevention' 'Prevention; Travel'
 'Symptoms' 'Prevention; Outbreak' 'Prevention: Travel'
 'Prevention; travel' 'Prevntion' 'Outbreak ' 'Vaccine, Travel'
 'Outbreak; Travel' 'outbreak' 'Travel; prevention' 'Travel; Symptoms'
 'Ountbreak' 'outbreak ' 'prevention' ' Outbreak; Prevention'
 ' Prevention' 'Outbrek; Symptoms' 'Outbreak; symptoms'
 'Outbreak; prevention' 'prevention; outbreak' 'Prevention; Outbreak '
 'Outbreak Prevention' 'outbreak, prevention' 'Prevention; outbreak'
 'outbreak; prevention' 'travel; prevention' 'Symptoms; Prevention'
 'Prevention, Outbreak' 'Outbreak; Awarness' 'Prevention; Awarness'
 'Symptoms, Awarness' 'Outbreak;Awareness' 'Outbreak; reaso

In [5]:
def clean_text(text):
    text = re.sub(r'RT\s@\w+:\s', '', text) # remove retweet text
    text = re.sub(r'https?:\/\/\S+', '', text) # remove URLs
    text = re.sub(r'&\w+;', '', text) # remove HTML entities
    text = re.sub(r'\d+', '', text) # remove numbers
    text = re.sub(r'\W+', ' ', text) # remove special characters
    text = text.lower() # convert to lowercase
    return text

In [ ]:
df_data["clean_text"] = df_data["text"].apply(clean_text)


# Filtering the rare terms.
df_data_filtered = df_data.groupby("Category").filter(lambda x: len(x) >1)
df_data_filtered.shape
print(df_data_filtered["Category"])
df_data_filtered["Category"]=[''.join(i.split()).lower() for i in df_data_filtered["Category"]]

2                  Outbreak
4                  Outbreak
12        Vaccine; Outbreak
28        Outbreak; Vaccine
39                 Outbreak
               ...         
487    Prevention; Awarness
488              Prevention
490                Symptoms
492              Prevention
496                Symptoms
Name: Category, Length: 938, dtype: object


In [ ]:
def filter_category(categories):
  #options = ['travel', 'outbreak','prevention','vaccine','symptoms']
  if ',' in categories:
    category_list = categories.split(',')
  elif ';' in categories:
    category_list = categories.split(';')
  else:
    category_list = [categories]

  print(category_list)
  '''
  temp = []
  for item in category_list:
    item = item.strip().lower()
    if 'awarness' == item:
      continue
    temp.append(item.strip())
  #print('temp =', temp)
  '''
  return category_list

In [ ]:
df_data_filtered["Category"] = df_data_filtered["Category"].apply(filter_category)

labels=[item for item in df_data_filtered["Category"]]

In [ ]:
df_data_filtered.columns

Index(['id', 'Account Created At(same for all rows)', 'status created_at',
       'Fav_cont', 'retweeted_count', 'reply_to_status', 'reply_to_screen',
       'quote', 'No', 'Category', 'text', 'Yes/No', 'Unnamed: 0',
       'clean_text'],
      dtype='object')

In [ ]:
# Convert String Labels into List of Strings:
df_data_filtered["Category"] = df_data_filtered["Category"].apply(
    lambda x: str(x).split(",")
)

df_data_filtered["Category"].values[:10]

array([list(["['outbreak']"]), list(["['outbreak']"]),
       list(["['vaccine'", " 'outbreak']"]),
       list(["['outbreak'", " 'vaccine']"]), list(["['outbreak']"]),
       list(["['outbreak'", " 'symptoms']"]),
       list(["['outbreak'", " 'vaccine']"]),
       list(["['outbreak'", " 'vaccine']"]),
       list(["['vaccine'", " 'outbreak']"]),
       list(["['vaccine'", " 'outbreak']"])], dtype=object)

In [ ]:
df_data_filtered["Category"][:10]

2                  [['outbreak']]
4                  [['outbreak']]
12     [['vaccine',  'outbreak']]
28     [['outbreak',  'vaccine']]
39                 [['outbreak']]
42    [['outbreak',  'symptoms']]
53     [['outbreak',  'vaccine']]
61     [['outbreak',  'vaccine']]
67     [['vaccine',  'outbreak']]
69     [['vaccine',  'outbreak']]
Name: Category, dtype: object

In [ ]:
# Perform 80-20 Ratio Split with Stratified Sampling Technique:
test_split = 0.2

# Initial train and test split.
train_df, test_df = train_test_split(
    df_data_filtered,
    test_size=test_split,
    stratify=df_data_filtered["Category"].values,
    random_state = 95
)

# Splitting the test set further into validation
# and new test sets.
val_df = test_df.sample(frac=0.5)
test_df.drop(val_df.index, inplace=True)

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 750
Number of rows in validation set: 94
Number of rows in test set: 84


In [ ]:
# Create a TensorFlow RaggedTensor for terms
terms = tf.ragged.constant(train_df["Category"].values)
lookup = tf.keras.layers.StringLookup(output_mode="multi_hot")
lookup.adapt(terms)
vocab = lookup.get_vocabulary()

# Define a function to invert multi-hot encoded labels

def invert_multi_hot(encoded_labels):
    """Reverse a single multi-hot encoded label to a tuple of vocab terms."""
    hot_indices = np.argwhere(encoded_labels == 1.0)[..., 0]
    return np.take(vocab, hot_indices)


print("Vocabulary:\n")
print(vocab)

Vocabulary:

['[UNK]', "['outbreak']", "['prevention']", "['travel']", " 'prevention']", "['outbreak'", " 'outbreak']", "['prevention'", "['travel'", "['vaccine'", " 'travel']", "['symptoms']", " 'awarness']", " 'symptoms']", "['vaccine']", " 'vaccine']"]
